In [33]:
import os
import pandas as pd
import numpy as np
from typing import Any, Type
from neo4j import Query, GraphDatabase, RoutingControl, Result 
from graphdatascience import GraphDataScience
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents import initialize_agent, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.messages import HumanMessage
import functools
import json

In [34]:
## Change this
os.environ['OPENAI_API_KEY'] = "yor key here" 
DB_ULR = "neo4j://localhost:7687"
DB_USER = "neo4j"
DB_PASS = "test1234"
DB_NAME = "movies" 

In [35]:
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS))
driver.verify_connectivity()

In [36]:
gds = GraphDataScience(driver)
gds.set_database(DB_NAME)
gds.version()

'2.6.2'

# Provide tools that LLM can use

In [37]:
class ActorModel(BaseModel):
    actor_name: str = Field(description="actor name")

def find_movies_given_an_actor(driver, actor_name: str) -> pd.DataFrame:
    """Given the name of an actor this function will find the movies"""
    return driver.execute_query(
        ''' 
            match (p:Person{name:$actor})-[r:ACTED_IN]->(m:Movie)
            return m.title as movie, 
                m.released as released, 
                m.tagline as tagline, 
                r.roles as roles
        ''',
        database_=DB_NAME,
        actor=actor_name,
        routing_=RoutingControl.READ,
        result_transformer_= lambda r: r.to_df(),
        limit = 20
    )

In [38]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)
name_to_functions = {
    'find_movies_given_an_actor': functools.partial(find_movies_given_an_actor, driver=driver),
}
tools = [
    find_movies_given_an_actor
]
functions = [convert_to_openai_function(t) for t in tools]
llm_with_tools = llm.bind_functions(functions=functions)


In [39]:
response = llm_with_tools.invoke([HumanMessage(
    content="""
        What movies did Tom Hanks act in? 
    """
)])

In [40]:
no_tool_available_response = llm_with_tools.invoke([HumanMessage(
    content="""
        What is the age of Tom Hanks? 
    """
)])

In [41]:
no_tool_available_response

AIMessage(content="I'm sorry, but I don't have access to real-time information such as the age of Tom Hanks. I can help you find information about his movies if you're interested.")

In [42]:
tool_call = response.additional_kwargs
function_name = tool_call['function_call']['name']
function_params = json.loads(tool_call['function_call']['arguments'])
print("function_name: ", function_name, "\nfunction_params: ", function_params)

function_name:  find_movies_given_an_actor 
function_params:  {'actor_name': 'Tom Hanks'}


In [43]:
df_acted_in = name_to_functions[function_name](**function_params)
df_acted_in.head()

,movie,released,tagline,roles
0,Apollo 13,1995,"Houston, we have a problem.",[Jim Lovell]
1,You've Got Mail,1998,At odds in life... in love on-line.,[Joe Fox]
2,A League of Their Own,1992,Once in a lifetime you get a chance to do some...,[Jimmy Dugan]
3,Joe Versus the Volcano,1990,"A story of love, lava and burning desire.",[Joe Banks]
4,That Thing You Do,1996,In every life there comes a time when that thi...,[Mr. White]
